In [1]:
from methods import *
import random
import matplotlib.pyplot as plt
import os

# 0 - Choose the protein structure you want to study

In [2]:
#structure_id = "1nd1" 
#file_name = "DATA//" + structure_id + ".pdb"
#protein = True
#structure_id = "4ldb"
#file_name = "DATA//" + structure_id + ".pdb"
#protein = True
#structure_id = "guanine"  #The code can be different if it is a molecule or a protein
#file_name = "DATA//" + structure_id + ".pdb"
#protein = False
structure_id = "4ogs"
file_name = "DATA//" + structure_id + ".pdb"
protein = True
#structure_id = "4pyi"
#file_name = "DATA//" + structure_id + ".pdb"
#protein = True
#structure_id = "6tpk"
#file_name = "DATA//" + structure_id + ".pdb"
#protein = True
#structure_id = "5ncx"
#file_name = "DATA//" + structure_id + ".pdb"
#protein = True

random.seed(27112000)

### 1 - Read the .pdb file and plot the 3D structure

In [3]:
if protein:
    structure = PDB_prot_reader(structure_id, file_name)
    print("Total number of carbon atoms: ", np.shape(structure[1])[0])
else:
    structure = PDB_mol_reader(structure_id, file_name)
    print("Total number of atoms: ", np.shape(structure[1])[0])

if os.path.exists("RESULTS//" + structure_id + ".gif"):
    print("Verify that this script has not already been run with this structure")
else:
    plot_3d(structure_id, structure[1])

Total number of carbon atoms:  450
Verify that this script has not already been run with this structure


c:\Users\hosca\anaconda3\envs\project\lib\site-packages\Bio\PDB\StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 4094.
  warnings.warn(
c:\Users\hosca\anaconda3\envs\project\lib\site-packages\Bio\PDB\StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 4183.
  warnings.warn(


### 2 - Analyse the 3D structure to determine the different values of epsilon

In [4]:
min = [np.min(idx) for idx in zip(*structure[1])]
max = [np.max(idx) for idx in zip(*structure[1])]

random_pair = [[random.randint(0, np.shape(structure[1])[0]-1), random.randint(0, np.shape(structure[1])[0]-1)]  for i in range(1000)]
dist = np.sort(np.array([np.linalg.norm(structure[1][idx[0]] - structure[1][idx[1]]) for idx in random_pair]))

while dist[0] == 0:
    dist = dist[1:]

min_dist, max_dist = np.min(dist), np.max(dist)
index = np.linspace(0, 900, 10, dtype = int)

""" if min_dist < 4:
    epsilon = [min_dist * 1.2, min_dist * 1.5, min_dist*2]
elif min_dist >= 4 and min_dist < 10:
    epsilon = [min_dist * 1.2, min_dist * 1.5, min_dist*2]
else:
    epsilon = [min_dist * 1.2, min_dist * 1.5, min_dist*2] """

epsilons = [4, 5, 6, 7, 8]

print(" Min (x,y,z): ", min,'\n',
    "Max (x,y,z): ", max, '\n',
    "Min euclidean distance: ", min_dist, '\n',
    "Max euclidean distance: ", max_dist, '\n',
    "Epsilon range: ", epsilons, '\n'
    " Distance batch: ", dist[index])

 Min (x,y,z):  [-13.024, 1.449, -1.645] 
 Max (x,y,z):  [40.887, 49.648, 54.151] 
 Min euclidean distance:  3.1755242 
 Max euclidean distance:  67.19296 
 Epsilon range:  [4, 5, 6, 7, 8] 
 Distance batch:  [ 3.1755242 14.241478  18.595028  22.334652  25.833252  28.843344
 32.145744  35.831085  40.186695  47.135895 ]


In [27]:
rips_complex = []
simplex_tree = []
total_inc = []
average_inc = []

for epsilon in epsilons:

    rips = generate_rips(structure[1], epsilon)
    edges = np.array([s[0] for s in rips.get_skeleton(1) if len(s[0])==2])
    triangles = np.array([s[0] for s in rips.get_skeleton(2) if len(s[0])==3])

    local, inc1, inc2 = inconsistency(structure, rips, edges, triangles)

    total_inc.append(inc1)
    average_inc.append(inc2)

    if os.path.exists("RESULTS//Rips_" + "_" + structure_id + "_" + str(epsilon) + ".gif"):
        print("Verify that this script has not already been run with this structure")
    else:
        plot_rips(structure[1], rips, edges, triangles, structure_id, epsilon, local)

Rips complex is of dimension 1 - 1955 simplices - 987 vertices.
Verify that this script has not already been run with this structure
Rips complex is of dimension 2 - 3194 simplices - 987 vertices.
Verify that this script has not already been run with this structure
Rips complex is of dimension 2 - 5271 simplices - 987 vertices.
Verify that this script has not already been run with this structure
Rips complex is of dimension 2 - 8876 simplices - 987 vertices.
Verify that this script has not already been run with this structure
Rips complex is of dimension 2 - 13835 simplices - 987 vertices.
Verify that this script has not already been run with this structure


In [28]:
total_inc = np.array(total_inc)
average_inc = np.array(average_inc)

print("The total inconsistency of the structure ", structure_id, " is ", total_inc)
print("The average inconsistency of the structure ", structure_id, " is ", average_inc)

res_total = ("RESULTS//" + structure_id + "_total.npy")
res_average = ("RESULTS//" + structure_id + "_average.npy")

np.save(res_total, total_inc)
np.save(res_average, average_inc)

The total inconsistency of the structure  4ldb  is  [2.14826946e-10 8.65247029e+02 1.21935085e+03 2.06587820e+03
 2.67978996e+03]
The average inconsistency of the structure  4ldb  is  [1.06649364e-11 2.84245179e+01 3.28004534e+01 4.25165257e+01
 4.78108592e+01]
